In [1]:
import numpy as np
from numba import njit

# References

* [Knapsack Problem Dynamic Programming](https://www.youtube.com/watch?v=zRza99HPvkQ)
* [Ekillion Train Example](https://github.com/takemaru/graphillion/wiki/Example-codes#ekillion---weighted-edges-and-vertices)
* [Finding All Knapsack Solutions (Paths) Within a Range of Weight Limits](https://github.com/takemaru/graphillion/issues/66)
* [Top-Down vs Bottom-Up Dynamic Programming](https://www.youtube.com/watch?v=M-NeO_9BU_A)

In [2]:
@njit
def knapsack(weights, values, capacity, n, lookup):
    """
    Based on these dynamic programming explanations:
    https://www.youtube.com/watch?v=hagBB17_hvg
    https://www.youtube.com/watch?v=zRza99HPvkQ

    Nearly identical code can be found here:
    https://www.geeksforgeeks.org/0-1-knapsack-problem-dp-10/

    Search for "Bottom-up Approach for 0/1 Knapsack Problem"

    Note that the weights and values vectors are assumed to have a zero
    prepended to them. Thus, the index of each item starts at one.
    """
    for i in range(n+1):
        for w in range(capacity+1):
            if i == 0 or w == 0:
                lookup[i, w] = 0
            elif weights[i] <= w:
                lookup[i, w] = max(
                    lookup[i-1, w-weights[i]] + values[i],
                    lookup[i-1, w]
                )
            else:
                lookup[i, w] = lookup[i-1, w]
    return lookup[n, w]

In [3]:
class KNAPSACK:
    def __init__(self, weights, values, capacity):
        self._n = len(weights)  # This is the actual number of items
        self._weights = np.concatenate(([0], weights))
        self._values = np.concatenate(([0], values))
        self._capacity = capacity
        self._lookup = np.empty((self._n+1, self._capacity+1), dtype=np.int64)
        self._max_value = None
        self._items = []

    def solve(self):
        knapsack(self._weights, self._values, self._capacity, self._n, self._lookup)

    @property
    def max_value(self):
        self._max_value = self._lookup[self._n, self._capacity]
        return self._max_value

    def get_value(self, idx):
        idx = [i+1 for i in idx]
        return self._values[idx].sum()

    @property
    def items(self):
        """
        These are the indices corresponding to the original weights or
        values vectors
        """
        self._items = []
        i = self._n
        j = self._capacity
        lookup = self._lookup.copy()
        while i > 0 and j > 0:
            if lookup[i, j] == lookup[i-1, j]:
                pass  # Item is excluded
            else:
                idx = i - 1
                self._items.append(idx)  # Item is included
                j = j - self._weights[i]
            i = i-1
        return self._items

In [277]:
weights = [1, 2, 3]
values = [6, 10, 12]
weights = [1, 2, 3, 2]
values = [6, 10, 12, 10]
capacity = 5

ks = KNAPSACK(weights, values, capacity)
ks.solve()
# assert ks.max_value == 220
# assert sorted(ks.items) == [1, 2]
# assert ks.get_value(ks.items) == 22

In [282]:
ks._lookup, ks.items

(array([[ 0,  0,  0,  0,  0,  0],
        [ 0,  6,  6,  6,  6,  6],
        [ 0,  6, 10, 16, 16, 16],
        [ 0,  6, 10, 16, 18, 22],
        [ 0,  6, 10, 16, 20, 26]]),
 [3, 1, 0])

In [283]:
26 - values[3]

16

In [284]:
16 - values[1]

6

In [279]:
def get_solutions(weights, i, j, lookup, partial):
    if i == 0:
        return None
    if j == 0:
        return partial.copy()

    solutions = []
    if lookup[i, j] == lookup[i-1, j]:
        pass  # Item is excluded
    else:
        new_solution = partial.copy()
        
        idx = i - 1
        # if idx not in new_solution:
        new_solution.append(idx)
        
        left = get_solutions(weights, i, j-weights[i-1], lookup, new_solution)
        if left is not None:
            solutions = solutions + left

        right = get_solutions(weights, i-1, j, lookup, partial)
        if right is not None:
            solutions.append(right)

        return solutions.copy()

    return get_solutions(weights, i-1, j, lookup, partial)

In [280]:
# weights = [10, 20, 30]
# values = [60, 100, 120]
# capacity = 50
get_solutions(ks._weights[1:], ks._n, ks._capacity, ks._lookup, [])

[3,
 1,
 0,
 [3, 0, 0, 0],
 [2, 1, [2, 0, 0], [1, 1, 0, [1, 0, 0, 0], [0, 0, 0, 0, 0]]]]

In [218]:
get_solutions(weights, ks._n, ks._capacity, ks._lookup, [])

[[[[3, 1, 0]], [[[[3, 0, 0, 0]]]]],
 [[[2, 1], [[[2, 0, 0]]]],
  [[[[1, 1, 0]], [[[[1, 0, 0, 0]]]]], [[[[[[0, 0, 0, 0, 0]]]]]]]]]

In [36]:
weights = [1, 2, 3, 2]
values = [6, 10, 12, 10]
capacity = 5

ks = KNAPSACK(weights, values, capacity)
ks.solve()

In [37]:
ks._lookup

array([[ 0,  0,  0,  0,  0,  0],
       [ 0,  6,  6,  6,  6,  6],
       [ 0,  6, 10, 16, 16, 16],
       [ 0,  6, 10, 16, 18, 22],
       [ 0,  6, 10, 16, 20, 26]])

In [38]:
ks.items

[3, 1, 0]

In [39]:
weights = [1, 2, 3, 2]
values = [6, 10, 12, 5]
capacity = 5

ks = KNAPSACK(weights, values, capacity)
ks.solve()
ks._lookup

array([[ 0,  0,  0,  0,  0,  0],
       [ 0,  6,  6,  6,  6,  6],
       [ 0,  6, 10, 16, 16, 16],
       [ 0,  6, 10, 16, 18, 22],
       [ 0,  6, 10, 16, 18, 22]])

In [40]:
ks.items

[2, 1]

In [41]:
weights = [1, 2, 3, 2]
values = [6, 10, 12, 6]
capacity = 5

ks = KNAPSACK(weights, values, capacity)
ks.solve()
ks._lookup

array([[ 0,  0,  0,  0,  0,  0],
       [ 0,  6,  6,  6,  6,  6],
       [ 0,  6, 10, 16, 16, 16],
       [ 0,  6, 10, 16, 18, 22],
       [ 0,  6, 10, 16, 18, 22]])

In [42]:
ks.items

[2, 1]

In [44]:
weights = [1,  2,  3, 2, 1]
values =  [6, 10, 12, 6, 5]
capacity = 5

ks = KNAPSACK(weights, values, capacity)
ks.solve()
ks._lookup

array([[ 0,  0,  0,  0,  0,  0],
       [ 0,  6,  6,  6,  6,  6],
       [ 0,  6, 10, 16, 16, 16],
       [ 0,  6, 10, 16, 18, 22],
       [ 0,  6, 10, 16, 18, 22],
       [ 0,  6, 11, 16, 21, 23]])

In [45]:
ks.items

[4, 2, 0]

# Version 1

In [258]:
def get_solutions(weights, i, j, lookup, partial):
    if i == 0:
        return []
    if j == 0:
        return partial.copy()

    solutions = []
    if lookup[i, j] == lookup[i-1, j]:
        pass  # Item is excluded
    else:
        new_solution = partial.copy()
        idx = i - 1
        new_solution.append(idx)
        left = get_solutions(weights, i, j-weights[i-1], lookup, new_solution)
        if len(left):
            solutions.append(left)

        right = get_solutions(weights, i-1, j, lookup, partial)
        if len(right):
            solutions.append(right)

        return solutions

    return get_solutions(weights, i-1, j, lookup, partial)

In [259]:
get_solutions(ks._weights[1:], ks._n, ks._capacity, ks._lookup, [])

[[[[3, 1, 0]], [[[[3, 0, 0, 0]]]]],
 [[[2, 1], [[[2, 0, 0]]]],
  [[[[1, 1, 0]], [[[[1, 0, 0, 0]]]]], [[[[[[0, 0, 0, 0, 0]]]]]]]]]

# Version 2

In [261]:
def get_solutions(weights, i, j, lookup, partial):
    if i == 0:
        return None
    if j == 0:
        return partial.copy()

    solutions = []
    if lookup[i, j] == lookup[i-1, j]:
        pass  # Item is excluded
    else:
        new_solution = partial.copy()
        
        idx = i - 1
        if idx not in new_solution:
            new_solution.append(idx)
        
        left = get_solutions(weights, i, j-weights[i-1], lookup, new_solution)
        if left is not None:
            solutions = solutions + left

        right = get_solutions(weights, i-1, j, lookup, partial)
        if right is not None:
            solutions.append(right)

        return solutions.copy()

    return get_solutions(weights, i-1, j, lookup, partial)

In [262]:
get_solutions(ks._weights[1:], ks._n, ks._capacity, ks._lookup, [])

[3, 1, 0, [3, 0], [2, 1, [2, 0], [1, 0, [1, 0], [0]]]]

# Version 3 (Graphillion)

In [1]:
weights = [1,  2,  3, 2, 1]
values =  [6, 10, 12, 6, 5]
capacity = 5

In [2]:
from graphillion import setset

items = list(range(len(values))) # item id (arbitrary)

cost_dic = {} # dictionary whose key is an item and whose value is
weight_dic = {}
for i in range(len(items)):
    cost_dic[items[i]] = values[i]
    weight_dic[items[i]] = weights[i]

setset.set_universe(items) # must be called

power_set = setset({}) # power set (all the combinations of the items)
weight_constrained_solutions = power_set.cost_le(weight_dic, capacity)

In [3]:
count = 0
for solution in weight_constrained_solutions.max_iter(cost_dic):
    print(solution)
    count += 1
    if count >= 3:
        break

{0, 2, 4}
{1, 2}
{0, 1, 3}


In [13]:
items = list(range(10_000))
setset.set_universe(items) # must be called

power_set = setset({}) # power set (all the combinations of the items)

In [14]:
cost_dic = {} # dictionary whose key is an item and whose value is
weight_dic = {}
for i in range(len(items)):
    cost_dic[items[i]] = 1
    weight_dic[items[i]] = 1

In [15]:
weight_constrained_solutions = power_set.cost_le(weight_dic, 100)

In [ ]:
count = 0
for solution in weight_constrained_solutions.max_iter(cost_dic):
    print(solution)
    count += 1
    if count >= 3:
        break

In [ ]:
# Testing the Limit

from graphillion import setset
import numpy as np

items = list(range(10_000))

cost_dic = {} # dictionary whose key is an item and whose value is
weight_dic = {}
for i in range(len(items)):
    cost_dic[items[i]] = np.random.randint(100)
    weight_dic[items[i]] = np.random.randint(100)

setset.set_universe(items) # must be called

power_set = setset({}) # power set (all the combinations of the items)

weight_constrained_solutions = power_set.cost_le(weight_dic, 100)

count = 0
for solution in weight_constrained_solutions.max_iter(cost_dic):
    print(solution)
    count += 1
    if count >= 3:
        break